In [1]:
import os
!pip install pyspark
!unzip -q /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=c226eb97c4e84156e1eadabbda4e938ed6ea252e5afdb125931433f0b08a21c0
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-adfsbimk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-adfsbimk
  Resolved https://github.com/huggingface/transformers.git to commit abaca9f9432a84cfaa95531de4c72334f38a42f2
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3

In [2]:
import pandas as pd
from pandasql import sqldf
from pyspark import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Spotify-data").getOrCreate()

df = spark.read\
        .option("header", "true") \
.csv('/kaggle/input/polifonia/automatic_EN_filtered.csv')
df.toPandas()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/08 11:55:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


,wn_synset,bn:id,pos,lemmata EN,definition,is it related to music?,note,removed EN
0,None,bn:00000627n,None,accelerando,A gradually increasing tempo of music,YES,None,None
1,None,bn:00000627n,None,halo,A gradually increasing tempo of music,YES,None,None
2,None,bn:00000627n,None,nightfall,A gradually increasing tempo of music,YES,None,None
3,None,bn:00000627n,None,accelenrando,A gradually increasing tempo of music,YES,None,None
4,None,bn:00000627n,None,lobsters,A gradually increasing tempo of music,YES,None,None
...,...,...,...,...,...,...,...,...
20573,wn:01050651v,bn:00086659v,v,descant,sing by changing register; sing by yodeling,YES,None,None
20574,wn:01256124n,bn:00081923n,n,yodeling,singing by changing back and forth between the...,YES,None,None
20575,wn:10803838n,bn:00081924n,n,yodeller,a singer who changes register rapidly (popular...,YES,None,None
20576,wn:04614844n,bn:00082041n,n,zill,one of a pair of small metallic cymbals worn o...,YES,None,None


In [3]:
from pyspark.sql.functions import col
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

context = r"""
sing by changing register; sing by yodeling
"""

result = question_answerer(question="What is a belonging lemmata?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Answer: 'sing by changing register', score: 0.451, start: 1, end: 26


In [4]:
df.groupby("bn:id").count().orderBy(col("count").desc()).toPandas()

,bn:id,count
0,bn:03858714n,85
1,bn:15428615n,70
2,bn:02431448n,57
3,bn:00016500n,55
4,bn:00124411n,54
...,...,...
4790,bn:00001237n,1
4791,bn:00008317n,1
4792,bn:00083265v,1
4793,bn:00114718r,1


In [5]:
df.groupby("definition").count().orderBy(col("count").desc()).toPandas()

,definition,count
0,Argentine tango is a musical genre and accompa...,85
1,The I–V–vi–IV progression is a common chord pr...,70
2,Music recording certification is a system of c...,57
3,A cassette that contains magnetic tape,55
4,Emo is a rock music genre characterized by an ...,54
...,...,...
4762,(music) the section of a movement (especially ...,1
4763,a chordophone that has a triangular frame cons...,1
4764,a computer program that performs a series of i...,1
4765,play on an instrument,1


In [6]:
!pip install --upgrade nltk
!pip list

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Package                                Version        Editable project location
-------------------------------------- -------------- -------------------------
absl-py                                1.4.0
accelerate                             0.21.0.dev0
access                                 1.1.9
affine                                 2.4.0
aiobotocore                            2.5.0
aiofiles                               22.1.0
aiohttp                                3.8.4
aiohttp-cors                  

In [7]:
from nltk.corpus import wordnet as wn
wn.ensure_loaded()


In [8]:
from nltk.corpus import wordnet as wn

wn
print(wn)
synsets = list(wn.all_eng_synsets())

#print(dir(wn))

<WordNetCorpusReader in '/usr/share/nltk_data/corpora/wordnet'>


In [9]:
df_def_list2 = list((df.select(['definition', "bn:id", "lemmata EN"]).
           toPandas().itertuples(index=False)))
print(df_def_list2[0])
df_def_list = list(df.select('definition').
           toPandas()['definition'])
wn_def_list = [it.definition() for it in synsets]

Pandas(definition='A gradually increasing tempo of music', _1='bn:00000627n', _2='accelerando')


In [10]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=78ff4a49095db506283b9f003473904b31b281efa68e9e67076f4bedae5a580e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [11]:
print(f"Len before none cleaining df_def_2: {len(df_def_list2)}")
df_def_list2 = list(filter(lambda item: item[0] is not None, df_def_list2))
print(f"Len after none cleaining df_def_2: {len(df_def_list2)}")

print(f"Len before none cleaining df_def: {len(df_def_list)}")
df_def_list = list(filter(lambda item: item is not None, df_def_list))
print(f"Len after none cleaining df_def: {len(df_def_list)}")

print(f"Len before none cleaining wn_def: {len(df_def_list)}")
wn_def_list = list(filter(lambda item: item is not None, wn_def_list))
print(f"Len after none cleaining wn_def: {len(df_def_list)}")

for i in range(0, len(df_def_list)):
    assert df_def_list[i] == df_def_list2[i][0]

Len before none cleaining df_def_2: 20578
Len after none cleaining df_def_2: 20576
Len before none cleaining df_def: 20578
Len after none cleaining df_def: 20576
Len before none cleaining wn_def: 20576
Len after none cleaining wn_def: 20576


In [12]:
from sentence_transformers import SentenceTransformer
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',)

df_tensors = model.encode(df_def_list, convert_to_tensor=True, device=device)
wn_tensor = model.encode(wn_def_list, convert_to_tensor=True, device=device)

Batches:   0%|          | 0/643 [00:00<?, ?it/s]

Batches:   0%|          | 0/3677 [00:00<?, ?it/s]

In [13]:
list(wn.all_eng_synsets())[0].definition()

"(usually followed by `to') having the necessary means or skill or know-how or authority to do something"

In [14]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
  StructField('df_lemmata', StringType(), True),
  StructField('bn:id', StringType(), True),
  StructField('wn:id', StringType(), True),
  StructField('df_definition', StringType(), True),
  StructField('proposition', StringType(), True),
  StructField('confidence', StringType(), True),
  ])
df2 = spark.createDataFrame([], schema)
print(df2)

DataFrame[df_lemmata: string, bn:id: string, wn:id: string, df_definition: string, proposition: string, confidence: string]


In [15]:
from sentence_transformers import util
outlist= []


for i in range(0, len(df_tensors)):
    res = util.pytorch_cos_sim(df_tensors[i], wn_tensor).tolist()[0]
    best_match_pos = res.index(max(res))
    #print(f"Original: {df_def_list[i]} | Best match: {wn_def_list[best_match_pos]}, with confidence {res[best_match_pos]}")
    outlist.append([df_def_list2[i][2], df_def_list2[i][1], f"wn:{synsets[best_match_pos].offset()}{synsets[best_match_pos].pos()}", df_def_list[i][0], wn_def_list[best_match_pos], res[best_match_pos]])

df2 = spark.createDataFrame(outlist, schema)
df2.write.csv("output.csv")